[![Notebook Tutorial](__code/__all/notebook_tutorial.png)](https://neutronimaging.pages.ornl.gov/en/tutorial/notebooks/water_intake_profile_calculator/)

# Select your IPTS 

In [1]:
from __code import system
from __code.water_intake_profile_calculator import WaterIntakeProfileCalculator, WaterIntakeProfileSelector

from __code.ui_builder import UiBuilder
o_builder = UiBuilder(ui_name = 'ui_water_intake_profile.ui')
from __code.roi_selection_ui import Interface

system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

pyuic5 /Users/j35/git/IPTS/python_notebooks/ui/ui_water_intake_profile.ui -o /Users/j35/git/IPTS/python_notebooks/__code/ui_water_intake_profile.py
** Using Debugging Mode! **


VBox(children=(HBox(children=(Label(value='IPTS-'), Text(value='', layout=Layout(width='10%')), Label(value='DOES NOT EXIST!', layout=Layout(width='20%'), _dom_classes=('result_label',)))), Label(value='OR'), HBox(children=(Label(value='Select Folder', layout=Layout(width='20%')), Select(layout=Layout(height='300px'), options=('IPTS-15177', 'IPTS-18813'), value='IPTS-15177'))), Button(button_style='info', description='HELP', style=ButtonStyle())))

# Description 

This notebook will calculate the water intake profile vs time of a sample.

Here are the steps (**bold** for user input/manipulation)
 * **select the normalized images**
 * images sorted by time
 * **region of interest seleted**
 * profile of counts vs vertical-pixel calculated (**select integrated algorithm: mean/median/sum**)
 * water intake profile vs file index or vs time
 * **export profiles**

# Python Import 

In [2]:
%gui qt

In [3]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from ipywidgets.widgets import interact
import numpy as np
import os
import ipywe.fileselector
from ipywidgets import widgets
from IPython.core.display import display, HTML
import pytz
import datetime

from NeuNorm.normalization import Normalization

from __code.metadata_handler import MetadataHandler
from __code import file_handler


# Select Images to Process

In [4]:
o_water = WaterIntakeProfileCalculator(working_dir = system.System.get_working_dir())
o_water.select_data()

Button(button_style='info', description='HELP', style=ButtonStyle())

HTML(value='Please wait...')

VBox(children=(Label(value='Select Images ...', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Users/j35/IPTS/IPTS-18813', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' .DS_Store                              |     Fri Feb  2 10:16:18 2018', ' Sample_2                               |     Directory', ' Sample_2_light                         |     Directory', ' Sample_2_light_timestamp_infos.txt     |     Wed Jan 31 09:30:55 2018', ' Sample_2_timestamp_infos.txt           |     Tue Jan 30 09:32:03 2018', ' data                                   |     Directory', ' shared                                 |     Directory'), value=()), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

HTML(value='Please wait...')

VBox(children=(Label(value='Select Images ...', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Users/j35/IPTS/IPTS-18813/Sample_2_light', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' 20180116_Sample2_Methane_150K_04Bar_0030_0441.tiff        |     Tue Jan 16 23:59:52 2018', ' 20180116_Sample2_Methane_150K_04Bar_0030_0442.tiff        |     Wed Jan 17 00:00:27 2018', ' 20180116_Sample2_Methane_150K_08Bar_0030_0431.tiff        |     Tue Jan 16 23:42:51 2018', ' 20180116_Sample2_Methane_150K_08Bar_0030_0432.tiff        |     Tue Jan 16 23:43:26 2018', ' 20180116_Sample2_Methane_150K_08Bar_0030_0433.tiff        |     Tue Jan 16 23:44:01 2018', ' 20180116_Sample2_Methane_150K_08Bar_0030_0434.tiff        |     Tue Jan 16 23:44:36 2018', ' 20180116_Sample2_Methane_150K_08Bar_0030_0435.tiff        |     Tue Jan 16 23:45:11 2018', ' 20180116_Sample2_Methane_150K_08Bar_0030_0436.tiff        |     Tue Jan 16 23:52:37 2018', ' 20180116_Sample2_Methane_150K_08Bar_0030_0437.tiff        |     Tue Jan 16 23:53:12 2018', ' 20180116_Sample2_Methane_150K_08Bar_0030_0438.tiff        |     Tue Jan 16 23:53:47 2018', ' 20180116_Sample2_Methane_150K_08Bar_0030_0439.tiff        |     Tue Jan 16 23:54:22 2018', ' 20180116_Sample2_Methane_150K_08Bar_0030_0440.tiff        |     Tue Jan 16 23:54:57 2018', ' 20180116_Sample2_Methane_150K_15Bar_0030_0426.tiff        |     Tue Jan 16 23:27:47 2018', ' 20180116_Sample2_Methane_150K_15Bar_0030_0427.tiff        |     Tue Jan 16 23:28:22 2018', ' 20180116_Sample2_Methane_150K_15Bar_0030_0428.tiff        |     Tue Jan 16 23:28:57 2018', ' 20180116_Sample2_Methane_150K_15Bar_0030_0429.tiff        |     Tue Jan 16 23:29:31 2018', ' 20180116_Sample2_Methane_150K_15Bar_0030_0430.tiff        |     Tue Jan 16 23:30:06 2018', ' 20180116_Sample2_Methane_150K_30Bar_0030_0421.tiff        |     Tue Jan 16 23:23:00 2018', ' 20180116_Sample2_Methane_150K_30Bar_0030_0422.tiff        |     Tue Jan 16 23:23:35 2018', ' 20180116_Sample2_Methane_150K_30Bar_0030_0423.tiff        |     Tue Jan 16 23:24:09 2018', ' 20180116_Sample2_Methane_150K_30Bar_0030_0424.tiff        |     Tue Jan 16 23:24:44 2018', ' 20180116_Sample2_Methane_150K_30Bar_0030_0425.tiff        |     Tue Jan 16 23:25:19 2018', ' 20180116_Sample2_Methane_150K_61Bar_0030_0416.tiff        |     Tue Jan 16 23:15:39 2018', ' 20180116_Sample2_Methane_150K_61Bar_0030_0417.tiff        |     Tue Jan 16 23:16:14 2018', ' 20180116_Sample2_Methane_150K_61Bar_0030_0418.tiff        |     Tue Jan 16 23:16:49 2018', ' 20180116_Sample2_Methane_150K_61Bar_0030_0419.tiff        |     Tue Jan 16 23:17:24 2018', ' 20180116_Sample2_Methane_150K_61Bar_0030_0420.tiff        |     Tue Jan 16 23:17:59 2018', ' 20180116_Sample2_Methane_150K_92Bar_0030_0410.tiff        |     Tue Jan 16 23:03:55 2018', ' 20180116_Sample2_Methane_150K_92Bar_0030_0411.tiff        |     Tue Jan 16 23:04:30 2018', ' 20180116_Sample2_Methane_150K_92Bar_0030_0412.tiff        |     Tue Jan 16 23:05:05 2018', ' 20180116_Sample2_Methane_150K_92Bar_0030_0413.tiff        |     Tue Jan 16 23:05:40 2018', ' 20180116_Sample2_Methane_150K_92Bar_0030_0414.tiff        |     Tue Jan 16 23:06:15 2018', ' 20180116_Sample2_Methane_150K_92Bar_0030_0415.tiff        |     Tue Jan 16 23:06:50 2018', ' 20180116_Sample2_Methane_200K_00Bar_0030_0296.tiff        |     Tue Jan 16 21:05:30 2018', ' 20180116_Sample2_Methane_200K_00Bar_0030_0297.tiff        |     Tue Jan 16 21:06:04 2018', ' 20180116_Sample2_Methane_200K_00Bar_0030_0298.tiff        |     Tue Jan 16 21:06:39 2018', ' 20180116_Sample2_Methane_200K_00Bar_0030_0299.tiff        |     Tue Jan 16 21:07:1

In [6]:
o_water.sort_and_load()

HBox(children=(Label(value='Retrieving Time Stamp', layout=Layout(width='20%')), IntProgress(value=0, layout=Layout(width='50%'), max=5)))

HBox(children=(Label(value='Loading sample', layout=Layout(width='20%')), IntProgress(value=0, max=5)))

# Select Profile Region 

In [4]:
#for debugging only
import numpy as np
data = []
for i in range(10):
    data.append(np.random.random((500,500)))
_dict_data = {'list_data': data}

In [5]:
o_gui = WaterIntakeProfileSelector(dict_data=_dict_data)
o_gui.show()

0
1
2
3
4
5
6
7
8
9
8
7
6
5
4
3
2
3
4
5
6
7
8
9
8
7
6
5
4
3
2
1
0
1
2
3
4
5
6
7
8
